In [1]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np

from recommenders.datasets.python_splitters import python_random_split
from recommenders.utils.constants import SEED
from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

2023-05-09 23:37:19.360739: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 23:37:19.383464: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-09 23:37:19.383942: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-09 23:37:19.891450: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
yaml_file = "./lightgcn.yaml"

In [12]:
EPOCHS = 50
BATCH_SIZE = 128
TOP_K = 10

# datacsv = pd.read_csv("data/user_recipe.csv")
# datacsv.columns = ["userID", "itemID", "rating", "modifyDate"]
datacsv_new = pd.read_csv("data/new_user_recipe.csv")
datacsv_new.columns = ["idx", "itemID","userID", "text", "rating"]
datacsv_new.drop(["idx"], inplace=True, axis=1)

# train, test = python_random_split(datacsv, 0.75)
train_new, test_new = python_random_split(datacsv_new, 0.75)
# data = ImplicitCF(train=train, test=test, seed=SEED)
data_new = ImplicitCF(train=train_new, test=train_new, seed=SEED)

/home/vscode/.local/lib/python3.9/site-packages/recommenders/models/deeprec/DataModel/ImplicitCF.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train if test is None else train.append(test)


In [13]:
# data.train_loader(model.batch_size)

In [14]:
# users, pos_items, neg_items = model.data.train_loader(model.batch_size)

In [15]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,
                         )

In [16]:
model = LightGCN(hparams, data=data_new, seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


2023-05-09 23:39:40.932085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 23:39:40.932373: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-09 23:39:40.932591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [17]:
with Timer() as train_time:
    model.fit()

Epoch 1 (train)8.1s: train loss = 0.62979 = (mf)0.62937 + (embed)0.00042
Epoch 2 (train)8.5s: train loss = 0.59511 = (mf)0.59422 + (embed)0.00089
Epoch 3 (train)8.5s: train loss = 0.54935 = (mf)0.54753 + (embed)0.00182
Epoch 4 (train)8.4s: train loss = 0.49177 = (mf)0.48851 + (embed)0.00325
Epoch 5 (train)8.5s + (eval)0.3s: train loss = 0.43424 = (mf)0.42929 + (embed)0.00495, recall = 0.00000, ndcg = 0.00000, precision = 0.00000, map = 0.00000
Epoch 6 (train)8.5s: train loss = 0.39006 = (mf)0.38334 + (embed)0.00672
Epoch 7 (train)8.4s: train loss = 0.35072 = (mf)0.34228 + (embed)0.00844
Epoch 8 (train)8.5s: train loss = 0.32167 = (mf)0.31158 + (embed)0.01010
Epoch 9 (train)8.4s: train loss = 0.29344 = (mf)0.28174 + (embed)0.01169
Epoch 10 (train)8.5s + (eval)0.3s: train loss = 0.27190 = (mf)0.25870 + (embed)0.01321, recall = 0.00000, ndcg = 0.00000, precision = 0.00000, map = 0.00000
Epoch 11 (train)8.4s: train loss = 0.25263 = (mf)0.23799 + (embed)0.01464
Epoch 12 (train)8.5s: train l

In [26]:
topk_scores = model.recommend_k_items(test_new, top_k=TOP_K, remove_seen=True)

topk_scores_max = topk_scores['prediction'].max()
topk_scores_min = topk_scores['prediction'].min()
topk_scores['prediction'] = (topk_scores['prediction'] - topk_scores_min) / (topk_scores_max - topk_scores_min)

0.0

In [36]:
eval_map = map_at_k(test_new, topk_scores, k=1)
eval_ndcg = ndcg_at_k(test_new, topk_scores, k=1)
eval_precision = precision_at_k(test_new, topk_scores, k=1)
eval_recall = recall_at_k(test_new, topk_scores, k=1)

print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

MAP:	0.004728
NDCG:	0.162500
Precision@K:	0.162500
Recall@K:	0.004332
